In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [22]:
data = pd.read_csv('dataset.csv')

def convert_to_np_array(s):
    decoded_list = []
    for number in s.replace('[','').replace(']','').split():
        decoded_list.append(int(number))
    return np.array(decoded_list)

data['decoded_word'] = data['decoded_word'].apply(convert_to_np_array)

numpy.ndarray